In [1]:
import sys
import pickle
sys.path.append("../tools/")
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from fancyimpute import KNN
from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.feature_selection import *

from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedShuffleSplit

with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

data_dict.pop('TOTAL',0) 
data_dict.pop('THE TRAVEL AGENCY IN THE PARK',0) 



enron = pd.DataFrame.from_records(list(data_dict.values()))
employees = pd.Series(list(data_dict.keys())) 
enron.set_index(employees, inplace=True) 
# Index is employees name

enron.replace(to_replace='NaN', value=np.nan, inplace=True)



enron.drop([u'email_address'] ,axis=1, inplace=True)

all_nans = enron[['from_poi_to_this_person','from_this_person_to_poi',
                  'from_messages','to_messages','shared_receipt_with_poi']].isnull().all(1)

enron.loc[all_nans, ['from_poi_to_this_person','from_this_person_to_poi',
                  'from_messages','to_messages','shared_receipt_with_poi']] = 0 


enron['ratio_from_poi'] = enron['from_poi_to_this_person']/enron['to_messages']
enron['ratio_from_poi'].fillna(0, inplace=True)



enron['ratio_to_poi'] = enron['from_this_person_to_poi']/enron['from_messages']
enron['ratio_to_poi'].fillna(0, inplace=True)

print(enron.columns)

enron_copy = pd.DataFrame(enron[[u'bonus', u'deferral_payments', u'deferred_income', u'director_fees',
       u'exercised_stock_options', u'expenses', u'from_messages',
       u'from_poi_to_this_person', u'from_this_person_to_poi',
       u'loan_advances', u'long_term_incentive', u'other', u'poi',
       u'restricted_stock', u'restricted_stock_deferred', u'salary',
       u'shared_receipt_with_poi', u'to_messages', u'total_payments',
       u'total_stock_value', u'ratio_from_poi', u'ratio_to_poi']].copy())

col = [[u'bonus', u'deferral_payments', u'deferred_income', u'director_fees',
       u'exercised_stock_options', u'expenses', u'from_messages',
       u'from_poi_to_this_person', u'from_this_person_to_poi',
       u'loan_advances', u'long_term_incentive', u'other', u'poi',
       u'restricted_stock', u'restricted_stock_deferred', u'salary',
       u'shared_receipt_with_poi', u'to_messages', u'total_payments',
       u'total_stock_value', u'ratio_from_poi', u'ratio_to_poi']]


enron = enron_copy.fillna(0)

enron.columns = col
enron.index = enron_copy.index

featurelist = [u'poi', u'bonus', u'deferral_payments', u'deferred_income', u'director_fees',
       u'exercised_stock_options', u'expenses', u'from_messages',
       u'from_poi_to_this_person', u'from_this_person_to_poi',
       u'loan_advances', u'long_term_incentive', u'other',
       u'restricted_stock', u'restricted_stock_deferred', u'salary',
       u'shared_receipt_with_poi', u'to_messages', u'total_payments',
       u'total_stock_value', u'ratio_from_poi', u'ratio_to_poi']

enronml = pd.DataFrame(enron[[u'poi', u'bonus', u'deferral_payments', u'deferred_income', u'director_fees',
       u'exercised_stock_options', u'expenses', u'from_messages',
       u'from_poi_to_this_person', u'from_this_person_to_poi',
       u'loan_advances', u'long_term_incentive', u'other',
       u'restricted_stock', u'restricted_stock_deferred', u'salary',
       u'shared_receipt_with_poi', u'to_messages', u'total_payments',
       u'total_stock_value', u'ratio_from_poi', u'ratio_to_poi']].copy())




enronml = enronml.to_dict(orient="index")
dataset = enronml

data = featureFormat(dataset, featurelist, sort_keys = True)
labels, features = targetFeatureSplit(data)



Index([u'bonus', u'deferral_payments', u'deferred_income', u'director_fees',
       u'exercised_stock_options', u'expenses', u'from_messages',
       u'from_poi_to_this_person', u'from_this_person_to_poi',
       u'loan_advances', u'long_term_incentive', u'other', u'poi',
       u'restricted_stock', u'restricted_stock_deferred', u'salary',
       u'shared_receipt_with_poi', u'to_messages', u'total_payments',
       u'total_stock_value', u'ratio_from_poi', u'ratio_to_poi'],
      dtype='object')


/Users/abhishekbabuji/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from sklearn.naive_bayes import GaussianNB

pca = PCA()
gnbf = GaussianNB()
steps = [('scaler', MinMaxScaler()),
         ('best', SelectKBest()),
         ('pca', pca),
         ('gnbf', gnbf)]

pipeline = Pipeline(steps)

parameters = [    
{
'best__k':[3],
'pca__n_components': [1,2]
},
{
'best__k':[4],
'pca__n_components': [1,2,3]
},
{
'best__k':[5],
'pca__n_components': [1,2,3,4]
},
{
'best__k':[6],
'pca__n_components': [1,2,3,4,5]
},
{
'best__k':[7],
'pca__n_components': [1,2,3,4,5,6]
},
{
'best__k':[8],
'pca__n_components': [1,2,3,4,5,6,7]
},
{
'best__k':[9],
'pca__n_components': [1,2,3,4,5,6,7,8]
},
{
'best__k':[10],
'pca__n_components': [1,2,3,4,5,6,7,8,9]
},
{
'best__k':[11],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10]
},
{
'best__k':[12],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11]
},
{
'best__k':[13],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12]
},
{
'best__k':[14],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13]
},
{
'best__k':[15],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
},
{
'best__k':[16],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
},
{
'best__k':[17],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
},
{
'best__k':[18],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
},
{
'best__k':[19],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
},
{
'best__k':[20],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
},
{
'best__k':[21],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
}
]

cv = StratifiedShuffleSplit(test_size=0.2, random_state=42)
gnbforf = GridSearchCV(pipeline, param_grid = parameters, cv=cv, scoring="f1")
gnbforf.fit(features, labels)

means = gnbforf.cv_results_['mean_test_score']
stds = gnbforf.cv_results_['std_test_score']


for mean, std, params in zip(means, stds, gnbforf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))

print "\n"
print(gnbforf.best_estimator_ )
print "\n"
print(gnbforf.best_score_)
print "\n"
print(gnbforf.best_params_)

feature_step = gnbforf.best_estimator_.named_steps['best']

# Get SelectKBest scores, rounded to 2 decimal places, name them "feature_scores"
feature_scores = ['%.2f' % elem for elem in feature_step.scores_ ]
# Get SelectKBest pvalues, rounded to 3 decimal places, name them "feature_scores_pvalues"
feature_scores_pvalues = ['%.3f' % elem for elem in  feature_step.pvalues_ ]
# Get SelectKBest feature names, whose indices are stored in 'skb_step.get_support',
# create a tuple of feature names, scores and pvalues, name it "features_selected_tuple"
features_selected_tuple=[(featurelist[i+1], feature_scores[i], feature_scores_pvalues[i]) for i in feature_step.get_support(indices=True)]

# Sort the tuple by score, in reverse order
features_selected_tuple = sorted(features_selected_tuple, key=lambda feature: float(feature[1]) , reverse=True)

# Print
print ' '
print 'Selected Features, Scores, P-Values'
print features_selected_tuple


/Users/abhishekbabuji/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.269 (+/-0.503) for {'best__k': 3, 'pca__n_components': 1}
0.292 (+/-0.534) for {'best__k': 3, 'pca__n_components': 2}
0.318 (+/-0.519) for {'best__k': 4, 'pca__n_components': 1}
0.295 (+/-0.474) for {'best__k': 4, 'pca__n_components': 2}
0.343 (+/-0.546) for {'best__k': 4, 'pca__n_components': 3}
0.244 (+/-0.467) for {'best__k': 5, 'pca__n_components': 1}
0.242 (+/-0.470) for {'best__k': 5, 'pca__n_components': 2}
0.319 (+/-0.513) for {'best__k': 5, 'pca__n_components': 3}
0.282 (+/-0.453) for {'best__k': 5, 'pca__n_components': 4}
0.235 (+/-0.432) for {'best__k': 6, 'pca__n_components': 1}
0.275 (+/-0.509) for {'best__k': 6, 'pca__n_components': 2}
0.369 (+/-0.596) for {'best__k': 6, 'pca__n_components': 3}
0.328 (+/-0.524) for {'best__k': 6, 'pca__n_components': 4}
0.302 (+/-0.512) for {'best__k': 6, 'pca__n_components': 5}
0.264 (+/-0.477) for {'best__k': 7, 'pca__n_components': 1}
0.286 (+/-0.556) for {'best__k': 7, 'pca__n_components': 2}
0.237 (+/-0.454) for {'best__k': 7, 'pca

In [3]:
pca = PCA()
steps = [('scaler', MinMaxScaler()),
         ('best', SelectKBest()),
         ('pca', pca),
         ('knn', KNeighborsClassifier())]

pipeline = Pipeline(steps)

parameters = [    
{
'best__k':[3],
'pca__n_components': [1,2],
'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10]
},
{
'best__k':[4],
'pca__n_components': [1,2,3],
'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10]
},
{
'best__k':[5],
'pca__n_components': [1,2,3,4],
'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10]
},
{
'best__k':[6],
'pca__n_components': [1,2,3,4,5],
'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10]
},
{
'best__k':[7],
'pca__n_components': [1,2,3,4,5,6],
'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10]
},
{
'best__k':[8],
'pca__n_components': [1,2,3,4,5,6,7],
'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10]
},
{
'best__k':[9],
'pca__n_components': [1,2,3,4,5,6,7,8],
'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10]
},
{
'best__k':[10],
'pca__n_components': [1,2,3,4,5,6,7,8,9],
'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10]
},
{
'best__k':[11],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10],
'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10]
},
{
'best__k':[12],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11],
'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10]
},
{
'best__k':[13],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12],
'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10]
},
{
'best__k':[14],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13],
'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10]
},
{
'best__k':[15],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14],
'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10]
},
{
'best__k':[16],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],
'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10]
},
{
'best__k':[17],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],
'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10]
},
{
'best__k':[18],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],
'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10]
},
{
'best__k':[19],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18],
'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10]
},
{
'best__k':[20],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19],
'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10]
},
{
'best__k':[21],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10]
}
]

cv = StratifiedShuffleSplit(test_size=0.2, random_state=42)
knnforf = GridSearchCV(pipeline, param_grid = parameters, cv=cv, scoring="f1")
knnforf.fit(features, labels)

means = knnforf.cv_results_['mean_test_score']
stds = knnforf.cv_results_['std_test_score']


for mean, std, params in zip(means, stds, knnforf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))

print "\n"
print(knnforf.best_estimator_ )
print "\n"
print(knnforf.best_score_)
print "\n"
print(knnforf.best_params_)

feature_step = knnforf.best_estimator_.named_steps['best']

# Get SelectKBest scores, rounded to 2 decimal places, name them "feature_scores"
feature_scores = ['%.2f' % elem for elem in feature_step.scores_ ]
# Get SelectKBest pvalues, rounded to 3 decimal places, name them "feature_scores_pvalues"
feature_scores_pvalues = ['%.3f' % elem for elem in  feature_step.pvalues_ ]
# Get SelectKBest feature names, whose indices are stored in 'skb_step.get_support',
# create a tuple of feature names, scores and pvalues, name it "features_selected_tuple"
features_selected_tuple=[(featurelist[i+1], feature_scores[i], feature_scores_pvalues[i]) for i in feature_step.get_support(indices=True)]

# Sort the tuple by score, in reverse order
features_selected_tuple = sorted(features_selected_tuple, key=lambda feature: float(feature[1]) , reverse=True)

# Print
print ' '
print 'Selected Features, Scores, P-Values'
print features_selected_tuple

0.303 (+/-0.394) for {'best__k': 3, 'pca__n_components': 1, 'knn__n_neighbors': 1}
0.208 (+/-0.451) for {'best__k': 3, 'pca__n_components': 2, 'knn__n_neighbors': 1}
0.160 (+/-0.392) for {'best__k': 3, 'pca__n_components': 1, 'knn__n_neighbors': 2}
0.160 (+/-0.392) for {'best__k': 3, 'pca__n_components': 2, 'knn__n_neighbors': 2}
0.253 (+/-0.454) for {'best__k': 3, 'pca__n_components': 1, 'knn__n_neighbors': 3}
0.309 (+/-0.548) for {'best__k': 3, 'pca__n_components': 2, 'knn__n_neighbors': 3}
0.073 (+/-0.295) for {'best__k': 3, 'pca__n_components': 1, 'knn__n_neighbors': 4}
0.080 (+/-0.320) for {'best__k': 3, 'pca__n_components': 2, 'knn__n_neighbors': 4}
0.062 (+/-0.249) for {'best__k': 3, 'pca__n_components': 1, 'knn__n_neighbors': 5}
0.225 (+/-0.485) for {'best__k': 3, 'pca__n_components': 2, 'knn__n_neighbors': 5}
0.073 (+/-0.295) for {'best__k': 3, 'pca__n_components': 1, 'knn__n_neighbors': 6}
0.040 (+/-0.240) for {'best__k': 3, 'pca__n_components': 2, 'knn__n_neighbors': 6}
0.07

0.000 (+/-0.000) for {'best__k': 16, 'pca__n_components': 5, 'knn__n_neighbors': 8}
0.000 (+/-0.000) for {'best__k': 16, 'pca__n_components': 6, 'knn__n_neighbors': 8}
0.000 (+/-0.000) for {'best__k': 16, 'pca__n_components': 7, 'knn__n_neighbors': 8}
0.000 (+/-0.000) for {'best__k': 16, 'pca__n_components': 8, 'knn__n_neighbors': 8}
0.000 (+/-0.000) for {'best__k': 16, 'pca__n_components': 9, 'knn__n_neighbors': 8}
0.000 (+/-0.000) for {'best__k': 16, 'pca__n_components': 10, 'knn__n_neighbors': 8}
0.000 (+/-0.000) for {'best__k': 16, 'pca__n_components': 11, 'knn__n_neighbors': 8}
0.000 (+/-0.000) for {'best__k': 16, 'pca__n_components': 12, 'knn__n_neighbors': 8}
0.000 (+/-0.000) for {'best__k': 16, 'pca__n_components': 13, 'knn__n_neighbors': 8}
0.000 (+/-0.000) for {'best__k': 16, 'pca__n_components': 14, 'knn__n_neighbors': 8}
0.000 (+/-0.000) for {'best__k': 16, 'pca__n_components': 15, 'knn__n_neighbors': 8}
0.040 (+/-0.240) for {'best__k': 16, 'pca__n_components': 1, 'knn__n_n

0.000 (+/-0.000) for {'best__k': 20, 'pca__n_components': 12, 'knn__n_neighbors': 10}
0.000 (+/-0.000) for {'best__k': 20, 'pca__n_components': 13, 'knn__n_neighbors': 10}
0.000 (+/-0.000) for {'best__k': 20, 'pca__n_components': 14, 'knn__n_neighbors': 10}
0.000 (+/-0.000) for {'best__k': 20, 'pca__n_components': 15, 'knn__n_neighbors': 10}
0.000 (+/-0.000) for {'best__k': 20, 'pca__n_components': 16, 'knn__n_neighbors': 10}
0.000 (+/-0.000) for {'best__k': 20, 'pca__n_components': 17, 'knn__n_neighbors': 10}
0.000 (+/-0.000) for {'best__k': 20, 'pca__n_components': 18, 'knn__n_neighbors': 10}
0.000 (+/-0.000) for {'best__k': 20, 'pca__n_components': 19, 'knn__n_neighbors': 10}
0.250 (+/-0.384) for {'best__k': 21, 'pca__n_components': 1, 'knn__n_neighbors': 1}
0.217 (+/-0.483) for {'best__k': 21, 'pca__n_components': 2, 'knn__n_neighbors': 1}
0.301 (+/-0.254) for {'best__k': 21, 'pca__n_components': 3, 'knn__n_neighbors': 1}
0.087 (+/-0.280) for {'best__k': 21, 'pca__n_components': 4,

In [4]:
from sklearn.linear_model import LogisticRegression

pca = PCA()

steps = [('scaler', MinMaxScaler()),
         ('best', SelectKBest()),
         ('pca', pca),
         ('logreg', LogisticRegression())]

pipeline = Pipeline(steps)

parameters = [    
{
'best__k':[3],
'pca__n_components': [1,2],
'logreg__C': [0.0001, 0.001, 0.01, 1, 10, 100]
},
{
'best__k':[4],
'pca__n_components': [1,2,3],
'logreg__C': [0.0001, 0.001, 0.01, 1, 10, 100]
},
{
'best__k':[5],
'pca__n_components': [1,2,3,4],
'logreg__C': [0.0001, 0.001, 0.01, 1, 10, 100]
},
{
'best__k':[6],
'pca__n_components': [1,2,3,4,5],
'logreg__C': [0.0001, 0.001, 0.01, 1, 10, 100]
},
{
'best__k':[7],
'pca__n_components': [1,2,3,4,5,6],
'logreg__C': [0.0001, 0.001, 0.01, 1, 10, 100]
},
{
'best__k':[8],
'pca__n_components': [1,2,3,4,5,6,7],
'logreg__C': [0.0001, 0.001, 0.01, 1, 10, 100]
},
{
'best__k':[9],
'pca__n_components': [1,2,3,4,5,6,7,8],
'logreg__C': [0.0001, 0.001, 0.01, 1, 10, 100]
},
{
'best__k':[10],
'pca__n_components': [1,2,3,4,5,6,7,8,9],
'logreg__C': [0.0001, 0.001, 0.01, 1, 10, 100]
},
{
'best__k':[11],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10],
'logreg__C': [0.0001, 0.001, 0.01, 1, 10, 100]
},
{
'best__k':[12],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11],
'logreg__C': [0.0001, 0.001, 0.01, 1, 10, 100]
},
{
'best__k':[13],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12],
'logreg__C': [0.0001, 0.001, 0.01, 1, 10, 100]
},
{
'best__k':[14],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13],
'logreg__C': [0.0001, 0.001, 0.01, 1, 10, 100]
},
{
'best__k':[15],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14],
'logreg__C': [0.0001, 0.001, 0.01, 1, 10, 100]
},
{
'best__k':[16],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],
'logreg__C': [0.0001, 0.001, 0.01, 1, 10, 100]
},
{
'best__k':[17],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],
'logreg__C': [0.0001, 0.001, 0.01, 1, 10, 100]
},
{
'best__k':[18],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],
'logreg__C': [0.0001, 0.001, 0.01, 1, 10, 100]
},
{
'best__k':[19],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18],
'logreg__C': [0.0001, 0.001, 0.01, 1, 10, 100]
},
{
'best__k':[20],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19],
'logreg__C': [0.0001, 0.001, 0.01, 1, 10, 100]
},
{
'best__k':[21],
'pca__n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
'logreg__C': [0.0001, 0.001, 0.01, 1, 10, 100]
}
]

cv = StratifiedShuffleSplit(test_size=0.2, random_state=42)
logregforf = GridSearchCV(pipeline, param_grid = parameters, cv=cv, scoring="f1")
logregforf.fit(features, labels)

means = logregforf.cv_results_['mean_test_score']
stds = logregforf.cv_results_['std_test_score']


for mean, std, params in zip(means, stds, logregforf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))

print "\n"
print(logregforf.best_estimator_ )
print "\n"
print(logregforf.best_score_)
print "\n"
print(logregforf.best_params_)

feature_step = logregforf.best_estimator_.named_steps['best']

# Get SelectKBest scores, rounded to 2 decimal places, name them "feature_scores"
feature_scores = ['%.2f' % elem for elem in feature_step.scores_ ]
# Get SelectKBest pvalues, rounded to 3 decimal places, name them "feature_scores_pvalues"
feature_scores_pvalues = ['%.3f' % elem for elem in  feature_step.pvalues_ ]
# Get SelectKBest feature names, whose indices are stored in 'skb_step.get_support',
# create a tuple of feature names, scores and pvalues, name it "features_selected_tuple"
features_selected_tuple=[(featurelist[i+1], feature_scores[i], feature_scores_pvalues[i]) for i in feature_step.get_support(indices=True)]

# Sort the tuple by score, in reverse order
features_selected_tuple = sorted(features_selected_tuple, key=lambda feature: float(feature[1]) , reverse=True)

# Print
print ' '
print 'Selected Features, Scores, P-Values'
print features_selected_tuple

0.000 (+/-0.000) for {'best__k': 3, 'pca__n_components': 1, 'logreg__C': 0.0001}
0.000 (+/-0.000) for {'best__k': 3, 'pca__n_components': 2, 'logreg__C': 0.0001}
0.000 (+/-0.000) for {'best__k': 3, 'pca__n_components': 1, 'logreg__C': 0.001}
0.000 (+/-0.000) for {'best__k': 3, 'pca__n_components': 2, 'logreg__C': 0.001}
0.000 (+/-0.000) for {'best__k': 3, 'pca__n_components': 1, 'logreg__C': 0.01}
0.000 (+/-0.000) for {'best__k': 3, 'pca__n_components': 2, 'logreg__C': 0.01}
0.040 (+/-0.240) for {'best__k': 3, 'pca__n_components': 1, 'logreg__C': 1}
0.073 (+/-0.295) for {'best__k': 3, 'pca__n_components': 2, 'logreg__C': 1}
0.220 (+/-0.470) for {'best__k': 3, 'pca__n_components': 1, 'logreg__C': 10}
0.222 (+/-0.369) for {'best__k': 3, 'pca__n_components': 2, 'logreg__C': 10}
0.249 (+/-0.447) for {'best__k': 3, 'pca__n_components': 1, 'logreg__C': 100}
0.249 (+/-0.447) for {'best__k': 3, 'pca__n_components': 2, 'logreg__C': 100}
0.000 (+/-0.000) for {'best__k': 4, 'pca__n_components': 1

In [8]:
dump_classifier_and_data(logregforf.best_estimator_, dataset, featurelist)